In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

from transformers import BertTokenizer, BertModel
import torch

import warnings
warnings.filterwarnings("ignore")


In [9]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------- ----------------------------- 2.6/10.1 MB 15.1 MB/s eta 0:00:01
   ----------------------- ---------------- 6.0/10.1 MB 14.8 MB/s eta 0:00:01
   ------------------------------------ --- 9.2/10.1 MB 15.0 MB/s eta 0:00:01
   ---------------------------------------- 10.1/10.1 MB 14.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/553.2 kB ? eta -:--:--
   --------------------------------------- 553.2/553.2 kB 19.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 13.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------------------- 2.9/2.9 MB 14.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [35]:
pip install torch torchvision torchaudio


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   --------------------- ------------------ 2.4/4.3 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dhanalaxmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dhanalaxmi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dhanalaxmi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
df=pd.read_csv("data.csv")
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [56]:
df.shape

(8013, 10)

In [3]:
# Keep only required columns
df1= df[['Review text', 'Ratings']]
df1.head()


,Review text,Ratings
0,"Nice product, good quality, but price is now r...",4
1,They didn't supplied Yonex Mavis 350. Outside ...,1
2,Worst product. Damaged shuttlecocks packed in ...,1
3,"Quite O. K. , but nowadays the quality of the...",3
4,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [5]:
df['Ratings'].unique()


array([4, 1, 5, 2, 3], dtype=int64)

In [6]:
#Creating Sentiment labels
def label_sentiment(rating):
    return 1 if rating >= 4 else 0

df['sentiment'] = df['Ratings'].apply(label_sentiment)


In [7]:
df.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings,sentiment
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4,1
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1,0
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1,0
5,Baji Sankar,Mind-blowing purchase,"Certified Buyer, Hyderabad",173.0,45.0,Oct 2018,Good quality product. Delivered on time.READ MORE,5,1
6,Flipkart Customer,Must buy!,"Certified Buyer, Doom Dooma",403.0,121.0,Jan 2020,BEST PURCHASE It is a good quality and is more...,5,1


In [8]:
#Text Cleaning and Normalization
#
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # remove special characters
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

df['clean_text'] = df['Review text'].apply(clean_text)


In [9]:
#Bag of Words(BoW)
bow_vectorizer = CountVectorizer(max_features=5000)
X_bow = bow_vectorizer.fit_transform(df['clean_text'])

y = df['sentiment']


In [10]:
#TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['clean_text'])


In [11]:
#Word2Vec
tokenized_reviews = [text.split() for text in df['clean_text']]

w2v_model = Word2Vec(
    sentences=tokenized_reviews,
    vector_size=100,
    window=5,
    min_count=2,
    workers=4
)

def get_w2v_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_w2v = np.array([
    get_w2v_vector(tokens, w2v_model)
    for tokens in tokenized_reviews
])


In [12]:
#Bert
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def bert_embeddings(text_list):
    embeddings = []

    for text in text_list:
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=128
        )

        with torch.no_grad():
            outputs = bert_model(**inputs)

        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)

    return np.array(embeddings)

X_bert = bert_embeddings(df['clean_text'][:500])  # limit for speed


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)


In [27]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": LinearSVC(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42)
}


In [46]:
results= {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    results[name] = f1
    
    print(f"\n{name}")
    print("F1-Score:", f1)
    print(classification_report(y_test, y_pred))



Logistic Regression
F1-Score: 0.9054054054054054
              precision    recall  f1-score   support

           0       0.79      0.68      0.73        28
           1       0.88      0.93      0.91        72

    accuracy                           0.86       100
   macro avg       0.84      0.80      0.82       100
weighted avg       0.86      0.86      0.86       100


Linear SVM
F1-Score: 0.896551724137931
              precision    recall  f1-score   support

           0       0.74      0.71      0.73        28
           1       0.89      0.90      0.90        72

    accuracy                           0.85       100
   macro avg       0.82      0.81      0.81       100
weighted avg       0.85      0.85      0.85       100


Random Forest
F1-Score: 0.8767123287671232
              precision    recall  f1-score   support

           0       0.69      0.64      0.67        28
           1       0.86      0.89      0.88        72

    accuracy                           0.82     

In [47]:
print("TF-IDF")
results_df = pd.DataFrame(
    results.items(), columns=["Model", "F1-Score"]
).sort_values(by="F1-Score", ascending=False)

results_df


TF-IDF


,Model,F1-Score
0,Logistic Regression,0.905405
1,Linear SVM,0.896552
2,Random Forest,0.876712


In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X_bow, y, test_size=0.2, random_state=42, stratify=y
)


In [31]:
results= {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    results[name] = f1
    
    print(f"\n{name}")
    print("F1-Score:", f1)
    print(classification_report(y_test, y_pred))



Logistic Regression
F1-Score: 0.9233027522935779
              precision    recall  f1-score   support

           0       0.80      0.44      0.57       310
           1       0.88      0.97      0.92      1293

    accuracy                           0.87      1603
   macro avg       0.84      0.71      0.74      1603
weighted avg       0.86      0.87      0.85      1603


Linear SVM
F1-Score: 0.916728624535316
              precision    recall  f1-score   support

           0       0.71      0.47      0.57       310
           1       0.88      0.95      0.92      1293

    accuracy                           0.86      1603
   macro avg       0.80      0.71      0.74      1603
weighted avg       0.85      0.86      0.85      1603


Random Forest
F1-Score: 0.921415270018622
              precision    recall  f1-score   support

           0       0.73      0.50      0.60       310
           1       0.89      0.96      0.92      1293

    accuracy                           0.87      

In [32]:
print("Bag of Words")
results_df = pd.DataFrame(
    results.items(), columns=["Model", "F1-Score"]
).sort_values(by="F1-Score", ascending=False)

results_df

Bag of Words


,Model,F1-Score
0,Logistic Regression,0.923303
2,Random Forest,0.921415
1,Linear SVM,0.916729


In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    X_w2v, y, test_size=0.2, random_state=42, stratify=y
)


In [34]:
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    f1 = f1_score(y_test, y_pred)
    results[name] = f1
    
    print(f"\n{name}")
    print("F1-Score:", f1)
    print(classification_report(y_test, y_pred))



Logistic Regression
F1-Score: 0.8929558011049724
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       310
           1       0.81      1.00      0.89      1293

    accuracy                           0.81      1603
   macro avg       0.40      0.50      0.45      1603
weighted avg       0.65      0.81      0.72      1603


Linear SVM
F1-Score: 0.892573402417962
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       310
           1       0.81      1.00      0.89      1293

    accuracy                           0.81      1603
   macro avg       0.40      0.50      0.45      1603
weighted avg       0.65      0.81      0.72      1603


Random Forest
F1-Score: 0.9068100358422939
              precision    recall  f1-score   support

           0       0.74      0.25      0.38       310
           1       0.85      0.98      0.91      1293

    accuracy                           0.84     

In [35]:
print("Word2Vec")
results_df = pd.DataFrame(
    results.items(), columns=["Model", "F1-Score"]
).sort_values(by="F1-Score", ascending=False)

results_df


Word2Vec


,Model,F1-Score
2,Random Forest,0.906810
0,Logistic Regression,0.892956
1,Linear SVM,0.892573


In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X_w2v, y, test_size=0.2, random_state=42, stratify=y
)

svm = LinearSVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print("F1-Score (Word2Vec + SVM):", f1_score(y_test, y_pred))


F1-Score (Word2Vec + SVM): 0.892573402417962


In [40]:

X_train, X_test, y_train, y_test = train_test_split(
    X_bert_scaled, y[:len(X_bert)], test_size=0.2, random_state=42
)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("F1-Score (BERT):", f1_score(y_test, y_pred))


F1-Score (BERT): 0.9054054054054054


In [43]:
svm = LinearSVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
print("F1-Score (Bert + SVM):", f1_score(y_test, y_pred))


F1-Score (Bert + SVM): 0.8873239436619719


For deployment, I selected TF-IDF with Logistic Regression as it achieved the highest F1-score while being computationally efficient, interpretable, and ideal for real-time sentiment analysis systems.

In [53]:
scaler = StandardScaler()
X_bert_scaled = scaler.fit_transform(X_bert)

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, random_state=42, stratify=y
)


lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
print("F1-Score (BERT):", f1_score(y_test, y_pred))

F1-Score (BERT): 0.9223512336719883


In [55]:
import joblib

joblib.dump(tfidf_vectorizer, "tfidf.pkl")
joblib.dump(lr, "sentiment_model.pkl")
print("Successful")


Successful
